# Imports

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from sqlalchemy import create_engine
import sqlite3

In [211]:
# functions

def request_soup(url_link):    
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}    
    page = requests.get( url, headers = headers)
    soup_obj = BeautifulSoup(page.text, 'html.parser')
    return( soup_obj )

# Data Requesting

##  Home Page Scraping

In [9]:
# all products url
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# headers for request
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}

# requesting
page = requests.get(url=url, headers=headers)

# instatiating bs4 object
soup = BeautifulSoup(page.text, 'html.parser')

In [48]:
# finding load more products element
p = soup.find('div', class_='load-more-products')

# all products
all_products = int(p.find('h2').get('data-total'))

# products per page
products_per_page = int(p.find('h2').get('data-items-shown'))

# rounding up numer of pages needed for web scraping
total_pages = np.ceil(all_products/products_per_page)


##  All products in Home Page Scraping

In [64]:
# creating a page with all products
url_all_prods = url + '?&offset=0&page-size={}'.format(int(total_pages*products_per_page))

all_prods = requests.get(url = url_all_prods, headers=headers)

In [187]:
soup = BeautifulSoup(all_prods.text, 'html.parser')#.get('li', class_='product-item')

# soup.find('li', class_ = 'product-item').find('a').get('href') #.get('item-link')  #.get('item-link') #, class_ = 'item-link')
# all find all products listed in homepage
products = soup.find_all('li', class_='product-item')

# get link to all projects
home_links = ['https://www2.hm.com' + link.find('a').get('href') for link in products ]

##  All products in Each Product Page

In [331]:
# resulting list of all products to scrap
links = []

for link in home_links:
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')

    # gets the links to all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    links_ul = []
    links_ul = [ 'https://www2.hm.com' + item.get('href') for item in products]
    links.extend(links_ul)

In [355]:
# defining base dataframe
df_prods = pd.DataFrame(columns=['product_id', 'color', 'style_id', 'color_id', 'link'])

for link in home_links:
    
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')

    # scrap all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    for product in products:
        
        #product it
        product_id = product.get('data-articlecode')

        # color
        color = product.get('data-color')
        
        # style id
        style_id = product_id[:-3]
        
        # style id
        color_id = product_id[-3:]

        # link
        link = 'https://www2.hm.com/en_us/productpage.{}.html'.format(product_id)

        df_temp = pd.DataFrame( {'product_id': product_id, 'color': color, 'style_id' :style_id, 'color_id' : color_id, 'link': link}, index = [0] )
        
        df_prods = pd.concat([df_prods, df_temp], axis = 0)


df_prods.reset_index(inplace = True, drop = True)

df_prods.drop_duplicates('product_id',inplace = True)

## Individual Scraping

In [374]:
soup = request_soup(df_prods.link[0])
df_prods.link[0]

'https://www2.hm.com/en_us/productpage.1024256001.html'

In [393]:
# soup.find('div', class_='ProductDescription-module--productDescription__2mqXe')
soup.find('div', class_='details parbase')
soup_text = soup.text

In [398]:
page = requests.get( df_prods.link[0], headers = headers)
soup = BeautifulSoup(page.text, 'html.parser')


In [406]:
import lxml

# soup = BeautifulSoup(page.text, 'lxml')




# TODO
try to use lxml, if doesn't work use selenium

In [326]:
# SAVING THIS FOR REFERENCE IN CASE NEEDED LATER

# soup.find_all('a', class_='swatch')#[0].get('title')
# elements = soup.find_all('a', class_='swatch')

# df = pd.DataFrame(columns=['product_id', 'color', 'style_id', 'color_id', 'link'])

# for element in elements:
#     # article number
#     product_id = element.get('href').split('.')[1]
    
#     # color
#     color = element.get('title')

#     # style id
#     style_id = product_id[:-3]
    
#     # style id
#     color_id = product_id[-3:]

#     # link
#     link = 'https://www2.hm.com/en_us/productpage.{}.html'.format(product_id)

#     df_temp = pd.DataFrame( {'product_id': product_id, 'color': color, 'style_id' :style_id, 'color_id' : color_id, 'link': link}, index = [0] )
    
#     df = pd.concat([df, df_temp], axis = 0)

# df.reset_index(inplace = True, drop = True)


In [329]:
#Art. No.,color,style_id,color_id,link,Fit,Composition,Price,date,texts,Cotton,Polyester,Elastane,Elasterell-P,Modal,Viscose
# pd.DataFrame( {'href': h_ref, 'color': color}, index = [0] )

df.link[0]

'https://www2.hm.com/en_us/productpage.1024256001.html'

In [254]:
soup.find('ul', class_="inputlist clearfix")


# Data Parsing

# Data Saving